
Example script: alpha filtration persistent homology and barcode plotting.

This file demonstrates an end-to-end pipeline for computing and plotting barcodes
from the alpha filtration of a planar point cloud.

# Pipeline overview

1. Sample points in $\mathbb R^2$.
2. Compute a Delaunay triangulation and take its simplices up to dimension 2.
3. Compute squared alpha values $\alpha^2(\sigma)$ for these simplices.
4. Produce a filtration order by sorting simplices by nondecreasing $\alpha^2$,
   with a deterministic tie break that preserves the face relation.
5. Compute persistent homology over $\mathbb F_2$ in this filtration.
6. Convert persistence pairs into a :class:`homolipop.barcodes.Barcode`.
7. Plot the barcode.

# Mathematical conventions

## Point cloud

Let

\begin{align}X = \{x_0,\dots,x_{N-1}\} \subset \mathbb R^2\end{align}

be a finite set of points. In this script, ``points`` is an array of shape ``(N, 2)``
with row ``points[i] = x_i``.

## Delaunay complex

Let $\mathrm{Del}(X)$ denote a Delaunay triangulation of $X$.
In $\mathbb R^2$, its maximal simplices are triangles.
We work with the simplicial complex $K$ consisting of all faces of those triangles,
truncated to dimension at most 2.

## Squared alpha values

For a simplex $\sigma$ with vertex set $V(\sigma) \subset X$,
define

\begin{align}\alpha^2(\sigma)
   =
   \min_{c \in \mathbb R^2} \max_{x\in V(\sigma)} \|x-c\|^2.\end{align}

This is the squared radius of the smallest enclosing ball of the vertices.
The function :func:`homolipop.alpha.alpha_values_squared` is assumed to compute these
values for all simplices of $K$ up to the prescribed maximal dimension.

## Alpha filtration

For $t \ge 0$, define the subcomplex

\begin{align}K_t = \{\sigma \in K \mid \alpha^2(\sigma) \le t\}.\end{align}

Face monotonicity

\begin{align}\tau \subseteq \sigma \implies \alpha^2(\tau) \le \alpha^2(\sigma)\end{align}

ensures that each $K_t$ is a simplicial complex and $(K_t)_{t\ge 0}$ is a filtration.

In practice, the filtration is represented by an ordering of the simplices
$\sigma_0,\dots,\sigma_{m-1}$ such that the sequence of values
$\alpha^2(\sigma_i)$ is nondecreasing and every face appears no later than its simplex.

## Persistent homology and barcodes

Fix the field $\mathbb F_2$. Let $K_i$ be the subcomplex generated by
the first $i+1$ simplices in the filtration order.
Persistent homology is the family of vector spaces

\begin{align}H_k(K_0;\mathbb F_2) \to H_k(K_1;\mathbb F_2) \to \cdots \to H_k(K_{m-1};\mathbb F_2),\end{align}

for $k \ge 0$, with structure maps induced by inclusions.
The routine :func:`homolipop.persistence.persistent_homology_field` returns the standard
birth-death pairing produced by boundary matrix reduction, and
:func:`homolipop.barcodes.barcodes_from_persistence` converts this into barcode intervals
in the filtration parameter.

## Numerical notes

The values $\alpha^2(\sigma)$ are stored in floating point arithmetic.
Sorting by these values is stable only up to floating point equality.
The filtration constructor is assumed to implement a deterministic tie break that preserves
the face relation, so that the resulting order is admissible for persistence.


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np

from homolipop.alpha import alpha_values_squared
from homolipop.barcodes import barcodes_from_persistence
from homolipop.delaunay import delaunay_triangulation
from homolipop.filtration import alpha_filtration_order
from homolipop.persistence import field_Fp, persistent_homology_field
from homolipop.plotting import plot_barcodes
from homolipop.simplices import build_complex


def main() -> None:
    """
    Compute alpha filtration barcodes for random planar points and plot them.

    The script

    - samples ``80`` points uniformly in the unit square
    - builds the Delaunay complex up to dimension 2
    - computes squared alpha values and derives a filtration order
    - computes persistent homology over :math:`\\mathbb F_2`
    - builds barcode intervals and plots them using Matplotlib

    Output
    ------
    A barcode plot window is displayed. The script also prints the number of paired and
    unpaired persistence events.
    """
    rng = np.random.default_rng(0)
    points = rng.random((80, 2))

    delaunay = delaunay_triangulation(points)
    complex_data = build_complex(delaunay.delaunay_simplices, max_dim=2)

    alpha = alpha_values_squared(points, delaunay.delaunay_simplices, max_dim=2)
    filtration = alpha_filtration_order(alpha, complex_data.all_simplices)

    persistence = persistent_homology_field(filtration.simplices, field=field_Fp(2))
    barcode = barcodes_from_persistence(filtration, persistence)

    plot_barcodes(barcode, title="Homolipop alpha filtration barcodes over F_2")
    plt.show()

    print("pairs:", len(persistence.pairs), "unpaired:", len(persistence.unpaired))


if __name__ == "__main__":
    main()